<a href="https://colab.research.google.com/github/t-nakatani/signate_stu22/blob/exp/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from google.colab import drive
drive.mount('/content/drive')
ROOT = "/content/drive/MyDrive/Colab Notebooks/signate/"

#学習用データと評価用データの読み込み
train = pd.read_csv(os.path.join(ROOT, "train.csv"))
test = pd.read_csv(os.path.join(ROOT, "test.csv"))

jobdic = dict(zip([1, 2, 3, 4], ['DataScientist', 'ML Engineer', 'Software Engineer', 'Consultant']))

Mounted at /content/drive


In [25]:
def highlight_duplicate(val):
    if val["jobflag"] not in [1, 2, 3, 4]:
        print('')
        return ['background-color: orange']*len(val)
    else:
        return ['background-color: white']*len(val) 

concat_train_test = pd.concat([train, test]).reset_index(drop=True)
duplicates = concat_train_test[concat_train_test["description"].duplicated(keep=False)].sort_values(by=["description","id"])

# Train/Testの両方に含まれているデータについて、Testデータの件数をカウントする
dup_between_train_and_test = 0
for flag in duplicates['description'].unique():
  tmp = duplicates[duplicates['description'] == flag]['jobflag']
  if tmp.isna().sum() > 0 and tmp.notna().sum() > 0:
    dup_between_train_and_test += tmp.isna().sum()
print(f"Duplication between Train and Test: {dup_between_train_and_test}")

duplicates.head(30).style.apply(highlight_duplicate, axis=1)

# Duplication between Train and Test: 27

Duplication between Train and Test: 27

















,id,description,jobflag
946,946,"Collaborate with internal Perficient Architects and customer stakeholders on requirements definition Contribute to the creation of technical specifications and/or Solutions Design Document(s) based on customer requirements Deliver high-quality technical solutions using the Adobe Experience Cloud framework within an Agile development process Deploy Adobe solutions and media tags (DoubleClick, Facebook, etc.) via Adobe Launch extensions or custom JavaScript Participate in QA/UAT testing activities to validate adherence to acceptance criteria Document final deliverable to support various customer hand-off and/or enablement activities.",3.000000
1707,1707,"Collaborate with internal Perficient Architects and customer stakeholders on requirements definition Contribute to the creation of technical specifications and/or Solutions Design Document(s) based on customer requirements Deliver high-quality technical solutions using the Adobe Experience Cloud framework within an Agile development process Deploy Adobe solutions and media tags (DoubleClick, Facebook, etc.) via Adobe Launch extensions or custom JavaScript Participate in QA/UAT testing activities to validate adherence to acceptance criteria Document final deliverable to support various customer hand-off and/or enablement activities.",nan
2158,2158,"Communicate with internal stakeholders to answer questions, address concerns, troubleshoot, and fix issues",nan
2330,2330,"Communicate with internal stakeholders to answer questions, address concerns, troubleshoot, and fix issues",nan
2642,2642,"Decompose user requirements into logical functions/components Design, develop, and maintain the front-end web applications of a scalable a nalytics s ystem Create ad hoc metrics and reports based on unique business needs Have solid understanding of d ata w arehouse development and optimization techniques",nan
3005,3005,"Decompose user requirements into logical functions/components Design, develop, and maintain the front-end web applications of a scalable a nalytics s ystem Create ad hoc metrics and reports based on unique business needs Have solid understanding of d ata w arehouse development and optimization techniques",nan
669,669,"Design and conduct surveys, interviews, and focus groups with representatives from global organizations to identify promising and best practices that could be replicated Develop and adhere to workplans for deliverables and work products Develop and implement stakeholder engagement and communications plans Generate communication materials and resources Synthesize findings into themes, presentations, and reports Demonstrate strong writing skills, organization skills, and high attention to detail and quality",4.000000
1086,1086,"Design and conduct surveys, interviews, and focus groups with representatives from global organizations to identify promising and best practices that could be replicated Develop and adhere to workplans for deliverables and work products Develop and implement stakeholder engagement and communications plans Generate communication materials and resources Synthesize findings into themes, presentations, and reports Demonstrate strong writing skills, organization skills, and high attention to detail and quality",4.000000
1096,1096,"Document progress and results of projects through case studies, report to directors, senior sponsors, project management and other creative methods Perform additional duties and projects as assigned by management",4.000000
2122,2122,"Document progress and results of projects through case studies, report to directors, senior sponsors, project management and other creative methods Perform additional duties and projects as assigned by management",nan


In [28]:
duplicates = concat_train_test[concat_train_test["description"].duplicated(keep=False)].sort_values(by=["id"])
duplicates

,id,description,jobflag
9,9,<li>Maintain and improve existing predictive m...,1.0
84,84,"<li>Be a courageous safety leader, adhere to a...",1.0
236,236,<li>Experience with marketing technology is a ...,4.0
317,317,<li>Develop new skills outside of comfort zone...,4.0
445,445,<li>Support product innovation by developing s...,1.0
...,...,...,...
2896,2896,"<li>Participate in the design, architecture, a...",NaN
2904,2904,<li>Represent products distributed and markete...,NaN
3001,3001,<li>Maintain and improve existing predictive m...,NaN
3005,3005,<li> Decompose user requirements into logical ...,NaN


train内に被り: 2つ目以降のtrainを除外  
train, testで被り: 除外なし，post-processingでラベル転移  
test内に被り: 除外なし．

In [62]:
ids_to_process = []
for flag in duplicates['description'].unique():
  ids = duplicates[duplicates['description'] == flag].id.tolist()

  contain_train = sum([id < len(train) for id in ids]) > 0
  if contain_train:
    ids_to_process.append(ids)

test_data_of_train = {} # post-processingでtestのラベルをtrainのラベルからカンニングする際に用いる
id_to_remove = [] # trainないに被りがある時は2つ目以降を除去
for ids in ids_to_process:
  train_id = ids[0] 
  for id in ids[1:]:
    if id < len(train): # if id == train_data
      id_to_remove.append(id)
    else: # if id == test_data
      test_data_of_train[id] = train_id
print(test_data_of_train)
print(id_to_remove)

{3001: 9, 2423: 236, 2259: 445, 1822: 507, 1852: 507, 2676: 511, 1700: 595, 2409: 673, 2775: 686, 1662: 744, 2149: 807, 2695: 852, 1630: 856, 1551: 858, 1673: 858, 2168: 858, 1707: 946, 2304: 973, 2076: 1033, 2736: 1043, 2122: 1096, 2352: 1150, 2191: 1173, 2077: 1277, 1568: 1302, 2301: 1441, 2842: 1498}
[959, 940, 1439, 907, 1086, 1379, 1038, 1050, 1444]
